In [ ]:
# !pip install folium
# !pip install geocoder
# !pip install chart_studio

In [ ]:
# !pip install ipywidgets
# !pip install geopy
# !pip install git+https://github.com/BoseCorp/py-googletrans.git --upgrade

In [ ]:
import pandas as pd
import numpy as np
import mysql.connector as conn
import statistics


import time

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly import tools
init_notebook_mode(connected=True)

import plotly.express as px

In [ ]:
pd.set_option('display.float_format', lambda x: '%18f' %x)

In [ ]:
dataset = pd.read_csv('C:/Users/user/INSAID/Project/events_data.csv')
dataset.head()

In [ ]:
dataset.columns

## Fetching data from other sources

In [ ]:
import mysql.connector

#this is to connect to mysql database
mydb = mysql.connector.connect(
  host="cpanel.insaid.co",
  user="student",
  password="student",
  database="Capstone1"
)

#declaring a cursor variable of database to hold data of a specific table
mycursor = mydb.cursor()

#mycursor variable holds the entire data from gender_age_train table
mycursor.execute("SELECT * FROM gender_age_train")

#using fetchall function to get data from cursor and populating in another variable
myresult = mycursor.fetchall()

#inserting table data from variable into a dataframe
gender_age_train_df = pd.DataFrame(myresult,columns=['device_id','Gender','Age','Age_Group'])

#mycursor variable holds the entire data from phone_brand_device_model table
mycursor.execute("SELECT * FROM phone_brand_device_model")

#using fetchall function to get data from cursor and populating in another variable, this overwrites data from previous load
myresult = mycursor.fetchall()

#inserting table data from variable into a dataframe
phone_brand_device_model_df = pd.DataFrame(myresult,columns=['device_id','Brand','Model'])

#printing sample data from the dataframes
print("Data from Gender_Age_Train Table")
print("---------------------------------")
print(gender_age_train_df.head())
print("**************************************")
print("\n")
print("Data from Phone_Brand_Device_Model Table")
print("-----------------------------------------")
print(phone_brand_device_model_df.head())
print("**************************************")


In [ ]:
gender_age_train_df.info()

In [ ]:
phone_brand_device_model_df.info()

In [ ]:
dataset.info()

- **Checking for missing values**

In [ ]:
dataset.isnull().sum()

In [ ]:
gender_age_train_df.isnull().sum()

In [ ]:
phone_brand_device_model_df.isnull().sum()

## Data Preprocessing

- **Imputing missing values in "device_id" in dataset** 

In [ ]:
#Preparing list of unique latitude values for the missing deviceid records.
latlon_df = dataset[dataset.device_id.isnull()]['latitude'].unique()

In [ ]:
latlon_df

In [ ]:
#For imputing Missing values in DeviceId, we've fetched unique latitude for the missing deviceid and found that for that latitude there is only one deviceid throughout the dataset.
#Hence, we'll be imputing missing deviceid values by replacing the deviceid which the same latitudeid shares
for latitude in latlon_df:
    devid = list(dataset[(dataset.latitude == latitude) & (dataset.device_id.notnull())].device_id)[0]
    print('Latitude is {} for DeviceID: {}'.format(latitude, devid))
    dataset.loc[dataset.latitude == latitude, 'device_id'] = devid

In [ ]:
dataset.isnull().sum()

- **Imputing Missing values in "State"**

In [ ]:
#State wise count before imputing missing state values
dataset[(dataset.state == 'WestBengal') | (dataset.state == 'Karnataka') | (dataset.state == 'Gujarat') | (dataset.state == 'Bihar') | (dataset.state == 'Punjab') | (dataset.state == 'Kerala')]['state'].value_counts()

In [ ]:
#For imputing the missing values in State, we'll first find all their respecitve city values
city_list = list(dataset[dataset.state.isnull()].city.unique())
city_list

In [ ]:
#Will loop over the above cities list, fetch it's relevant state value where state value is not null and then populate that to all the records where city value is matching with the city value from our list
for city in city_list:
    state = list(dataset[(dataset.city == city) & (dataset.state.notnull())].state)[0]
    print("State is {} for City {}".format(state,city))
    dataset.loc[dataset.city == city, 'state'] = state

## Imputing missing values in "Latitude & Longitude"

In [ ]:
#checking if latitude and longitude are null for the same set of records
dataset[(dataset.latitude.isnull()) & (dataset.longitude.isnull())].count()

- We can see that the final count for the records where lat and long both are null is equal to the total number of null values.

In [ ]:
#Check for missing values in events_data_set
dataset.isnull().sum()

- Taking the ids and checking the counts for city and state

In [ ]:
device_id = dataset[(dataset.latitude.isnull()) & (dataset.longitude.isnull())]['device_id'].unique()

print("Total records : ", len(device_id))
print("---------------------------------------------------")
for ids in device_id:
    print("The total count for device_id : {} is : {}".format(ids, dataset[dataset.device_id == ids][['city','state']].nunique()))
# dataset[dataset.device_id == -8790560034584249344.000000][['city','state']].nunique()

- Each device_id belongs to one pair of city and state

In [ ]:
dataset[(dataset.latitude.isnull()) & (dataset.longitude.isnull())].sort_values(['device_id'])['device_id'].value_counts()

In [ ]:
#checking if lat and long values exist for atleast one record for the particular device_id
temp = dataset[~(dataset.latitude.isnull() & dataset.longitude.isnull())]
for ids in device_id:
    print("The device_id : {} lat : {} long : {}".format(ids, temp[temp.device_id == ids].latitude[:1].values[0], temp[temp.device_id == ids].longitude[:1].values[0]))

- **It seems that for all device_id lat and long values are already present in some of records, these can be used to impute the missing values at other places**

In [ ]:
# Replacing the missing values

for ids in device_id:
    dataset.loc[dataset.device_id==ids, 'latitude'] = temp[temp.device_id == ids].latitude[:1].values[0]
    dataset.loc[dataset.device_id==ids, 'longitude'] = temp[temp.device_id == ids].longitude[:1].values[0]
#     print("The device_id : {} lat : {} long : {}".format(ids, temp[temp.device_id == ids].latitude[:1].values[0], temp[temp.device_id == ids].longitude[:1].values[0]))

In [ ]:
dataset.isnull().sum()

- **Handling non-english characters in phone_brand_device_model_df dataset**

In [ ]:
# gender_age_train_df.brand.value_counts()
phone_brand_device_model_df.Brand.unique()

In [ ]:
# #Updating Mobile Brand for couple of Brands where English mapping is provided by INSAID
# lang_dict = {'华为' : 'Huawei', 
#              '小米' : 'Xiaomi',
#              '三星' : 'Samsung',
#              '魅族' : 'Meizu',
#              '酷派' : 'Coolpad',
#              '乐视' : 'LeEco',
#              '联想' : 'Lenovo'}

# phone_brand_device_model_df.Brand = phone_brand_device_model_df.Brand.apply(lambda x: lang_dict.setdefault(x,x))

In [ ]:

#To replace Non-English Characters in Brand column of phone_brand_device_model dataframe
#We are using a dictionary found online which has mapping for all Non-English Characters to English 
phone_brands_mapping = {"三星": "samsung","天语": "Ktouch", "海信": "hisense", "联想": "lenovo", "欧比": "obi",
                                "爱派尔": "ipair", "努比亚": "nubia", "优米": "youmi", "朵唯": "dowe", "黑米": "heymi",
                                "锤子": "hammer", "酷比魔方": "koobee", "美图": "meitu", "尼比鲁": "nibilu", "一加": "oneplus",
                                "优购": "yougo", "诺基亚": "nokia", "糖葫芦": "candy", "中国移动": "ccmc", "语信": "yuxin",
                                "基伍": "kiwu", "青橙": "greeno", "华硕": "asus", "夏新": "panasonic", "维图": "weitu",
                                "艾优尼": "aiyouni", "摩托罗拉": "moto", "乡米": "xiangmi", "米奇": "micky", "大可乐": "bigcola",
                                "沃普丰": "wpf", "神舟": "hasse", "摩乐": "mole", "飞秒": "fs", "米歌": "mige", "富可视": "fks",
                                "德赛": "desci", "梦米": "mengmi", "乐视": "lshi", "小杨树": "smallt", "纽曼": "newman",
                                "邦华": "banghua", "E派": "epai", "易派": "epai", "普耐尔": "pner", "欧新": "ouxin", "西米": "ximi",
                                "海尔": "haier", "波导": "bodao", "糯米": "nuomi", "唯米": "weimi", "酷珀": "kupo", "谷歌": "google",
                                "昂达": "ada", "聆韵": "lingyun", "小米": "Xiaomi", "华为": "Huawei", "魅族": "Meizu", "中兴": "ZTE",
                                "酷派": "Coolpad", "金立": "Gionee",
                                "索尼" : "Sony", "欧博信" : "Opssom", "奇酷" : "Qiku",
                                "酷比" : "CUBE", "康佳" : "Konka", "亿通" : "Yitong", "金星数码" : "JXD", "至尊宝" : "Monkey King",
                                "百立丰" : "Hundred Li Feng", "贝尔丰" : "Bifer", "百加" : "Bacardi", "诺亚信" : "Noain", 
                                "广信" : "Kingsun", "世纪天元" : "Ctyon", "青葱" : "Cong", "果米" : "Taobao", "斐讯" : "Phicomm",
                                "长虹" : "Changhong", "欧奇" : "Oukimobile", "先锋" : "XFPLAY", "台电" : "Teclast", "大Q" : "Daq",
                                "蓝魔" : "Ramos", "奥克斯" : "AUX", "索尼" : "Sony", "欧博信" : "Opssom", "奇酷" : "Qiku",
                                "酷比" : "CUBE", "康佳" : "Konka", "亿通" : "Yitong", "金星数码" : "JXD", "至尊宝" : "Monkey King",
                                "百立丰" : "Hundred Li Feng", "贝尔丰" : "Bifer", "百加" : "Bacardi", "诺亚信" : "Noain",
                                "广信" : "Kingsun", "世纪天元" : "Ctyon", "青葱" : "Cong", "果米" : "Taobao", "斐讯" : "Phicomm",
                                "长虹" : "Changhong", "欧奇" : "Oukimobile", "先锋" : "XFPLAY", "台电" : "Teclast", "大Q" : "Daq", 
                                "蓝魔" : "Ramos", "奥克斯" : "AUX", "飞利浦": "Philips", "智镁": "Zhimei", "惠普": "HP",
                                "原点": "Origin", "戴尔": "Dell", "碟米": "Diemi", "西门子": "Siemens", "亚马逊": "Amazon",
                                "宏碁": "Acer", "E人E本":"Eben",
                                '世纪星': "UB1", '丰米': "UB2", '优语':'UB3', '凯利通': "UB4", '唯比': "UB5", '嘉源': "UB6",
                                 '大显': "UB7", '天宏时代': "UB8", '宝捷讯': 'UB9','帷幄': 'UB10', '德卡诺': 'UB11',
                                '恒宇丰': 'UB12', '本为': 'UB13', '极米': 'UB14', '欧乐迪': 'UB15', '欧乐酷': 'UB16',
                                '欧沃': 'UB17', '瑞米': 'UB18', '瑞高': 'UB19', '白米': 'UB20', '虾米': 'UB21', '赛博宇华': 'UB22',
                                '首云': 'UB23', '鲜米': 'UB24'}

phone_brand_device_model_df.Brand = phone_brand_device_model_df.Brand.apply(lambda x: phone_brands_mapping.setdefault(x.strip(),x.strip()))
print("\n After mapping Non-English Characters in Brand: \n",phone_brand_device_model_df.Brand.unique())

- **Filtering out the initial dataFrame (dataset) to keep values of only 6 concerned States**

In [ ]:
dataset_filtered = dataset[(dataset.state == 'WestBengal') | (dataset.state == 'Karnataka') | (dataset.state == 'Gujarat') |  (dataset.state == 'Bihar') | 
         (dataset.state =='Punjab') | (dataset.state == 'Kerala')]

In [ ]:
dataset_filtered.isnull().sum()

In [ ]:
dataset_filtered.info()

- **Checking if the lat and long values are proper**

In [ ]:
fig = px.scatter_mapbox(dataset_filtered, lat="latitude", lon="longitude", zoom=3,hover_name='city')
fig.update_layout(mapbox_style="open-street-map")  # <== Using Mapbox
fig.show()

- **Merging Dataframes on device_id to create a single dataframe which contains customer age, gender, mobile brand and model**

In [ ]:
#Joining events dataset of 6 states & gender_age dataset on device_id
device_data_df=pd.merge(dataset_filtered,gender_age_train_df,on='device_id',how='inner') #performing inner join to fetch only matching device_id records
device_data_df.head()

In [ ]:
#Joining phone_brand_model dataset with gender_age dataset and events dataset for 6 states on device_id
device_data_df=pd.merge(device_data_df,phone_brand_device_model_df,on='device_id',how='inner') #performing inner join to fetch only matching device_id records
device_data_df.head()

In [ ]:
# for cols in device_data_df.columns:
#     print(cols)
#     print(device_data_df.eval(cols).unique())
#     print("--------------------")


device_data_df.columns
device_data_df.Age_Group.unique()


In [ ]:
device_data_df.eval(list(cols)).unique()

In [ ]:
a = str('abc')
a
eval("[str(a)]")
eval("[a]")

In [ ]:
type(a)

In [ ]:
device_data_df.eval(cols)

----------------------

In [ ]:
dataset_filtered.sample(5)

In [ ]:
dataset_filtered.info()

In [ ]:
dataset_filtered.device_id.nunique()

- **Analysing the dataset. Checking for null values**

In [ ]:
dataset_filtered.isna().sum()

- **We have missing values:**
   - device_id = 48
   - longitude = 42
   - latitude  = 42
   
Notes : Cities and States are not missing for any of the records, hence, Lat and Long values can be derived. So, these values are not a problem, but, Device_ids are missing, this could subsequently lead to loss of info.

- **Dealing with device_id**

In [ ]:
print("The percentage of missing values : {:0.2f}%".format(dataset_filtered.device_id.isna().sum()/dataset_filtered.event_id.count()*100))

- **This is a negligible percentage of missing values to deal with. We can safely drop it.**

In [ ]:
dataset_filtered = dataset_filtered.dropna(subset=['device_id'], how="any")

dataset_filtered.shape

- **Dealing with missing Lat and Long values**

Values having probs

In [ ]:
a = dataset_filtered[dataset_filtered.longitude.isna() & dataset_filtered.latitude.isna()][['device_id', 'city', 'state', 'event_id']].sort_values('device_id')

print("The total records displayed below: {}".format(len(a)))
print("*******************************************************")
print(a)

In [ ]:
dataset_filtered[dataset_filtered.longitude.isna() & dataset_filtered.latitude.isna()]['device_id'].value_counts()

---------
## Basic Maps

In [ ]:
a = dataset_filtered[(dataset_filtered.city == 'Araria') & (dataset_filtered.state == 'Bihar')]
print("The mean value for Araria, Bihar : Latitude = {0:0.6f},  Longitutde = {1:0.6f}".format(statistics.mean(a['latitude'].dropna()),
                                                                                  statistics.mean(a['longitude'].dropna())))
print()
a = dataset_filtered[(dataset_filtered.city == 'Moga') & (dataset_filtered.state == 'Punjab')]
print("The mean value for Araria, Bihar : Latitude = {0:0.6f},  Longitutde = {1:0.6f}".format(statistics.mean(a['latitude'].dropna()),
                                                                                  statistics.mean(a['longitude'].dropna())))

print()
a = dataset_filtered[(dataset_filtered.city == 'Bagaha') & (dataset_filtered.state == 'Bihar')]
print("The mean value for Araria, Bihar : Latitude = {0:0.6f},  Longitutde = {1:0.6f}".format(statistics.mean(a['latitude'].dropna()),
                                                                                  statistics.mean(a['longitude'].dropna())))

In [ ]:
#Replacing the missing values with mean of all available values
long = {'Araria' : 87.578218,
       'Moga' : 75.189438,
        'Bagaha' : 84.147561}

lat = {'Araria' : 26.209851,
       'Moga' : 30.895271,
       'Bagaha' : 27.163957}

# dataset_filtered

In [ ]:
temp = dataset_filtered[dataset_filtered.longitude.isna() & dataset_filtered.latitude.isna()]['city']
a = temp.apply(lambda x: lat[x])

for indices in a.index:
    dataset_filtered.loc[indices, 'latitude'] = a[indices]
    
a = temp.apply(lambda x: long[x])

for indices in a.index:
    dataset_filtered.loc[indices, 'longitude'] = a[indices]

In [ ]:
folium.Marker?

In [ ]:

# # geocode address and place marker on map

# # map
# map_zoo = folium.Map(location=[32.744524, -117.150302], zoom_start=14)

# # get location information for address
# address = geocoder.osm('2920 Zoo Dr, San Diego, CA 92101')

# # address latitude and longitude
# address_latlng = [address.lat, address.lng]

# # add marker to map
# folium.Marker(address_latlng, popup='San Diego Zoo', tooltip='click').add_to(map_zoo)

# # display map
# map_zoo

In [ ]:
a = dataset_filtered[dataset_filtered.state=='Bihar']
len(a)

In [ ]:
# # multiple markers using dataframe
# # there is an example below using apply function instead of loop
# start_time = time.time()
# # create map
# map_locations = folium.Map(location=[38, 80], zoom_start=4)

# # plot airport locations
# for (index, row) in a.iterrows():
#     folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], 
#                   popup= row.loc['city'] + ' ' + row.loc['state'], 
#                   tooltip='click').add_to(map_locations)

# total_time = time.time() - start_time
# # display map    
# map_locations
# # print("The total time taken for computation - ", total_time)

In [ ]:
dataset_filtered.head()

In [ ]:
dataset_filtered.isna().sum()

In [ ]:
dataset_filtered.city.nunique()

In [ ]:
dataset_filtered.latitude.nunique()

In [ ]:
!pip install googletrans